In [ ]:
import wbdata
import pandas as pd
from datetime import datetime
import os
import tkinter as tk
from tkinter import filedialog


def get_file_path(title, action="open", default_extension=".xlsx"):
    root = tk.Tk()
    root.withdraw()
    if action == "open":
        return filedialog.askopenfilename(
            title=title,
            filetypes=[("Excel files", "*.xlsx *.xls")]
        )
    else:
        return filedialog.asksaveasfilename(
            title=title,
            defaultextension=default_extension,
            filetypes=[("Excel files", "*.xlsx")]
        )

def download_indicators(indicators_dict, countries, dates):
    """Try to download each indicator and return successful ones"""
    available_data = []
    failed_indicators = []
    
    # Try each indicator individually
    for code, name in indicators_dict.items():
        try:
            indicator = {code: name}
            df = wbdata.get_dataframe(indicator, country=countries, date=dates)
            if not df.empty:
                available_data.append(df)
            else:
                failed_indicators.append((code, name))
        except:
            failed_indicators.append((code, name))
            print(f"Could not download indicator: {code} - {name}")
            continue
    
    # Combine all successful downloads if any
    if available_data:
        return pd.concat(available_data, axis=1), failed_indicators
    else:
        return None, failed_indicators
    
# Prompt user to select files
print("Please select the countries Excel file...")
countries_file = get_file_path("Select Countries Excel File")
print("Please select the indicators Excel file...")
indicators_file = get_file_path("Select Indicators Excel File")

# Read the Excel files
countries_df = pd.read_excel(countries_file)
indicators_df = pd.read_excel(indicators_file)

# Convert countries to list
countries = countries_df.iloc[:, 0].tolist()  # Take first column

# Convert indicators to dictionary
requested_indicators = dict(zip(indicators_df['indicator_code'], indicators_df['indicator_name']))


# Specify dates
dates = (datetime(2000, 1, 1), datetime(2024, 1, 1))

print("\nAttempting to download data...")
wb_data, failed_indicators = download_indicators(requested_indicators, countries, dates)

if failed_indicators:
    print("\nThe following indicators could not be downloaded:")
    for code, name in failed_indicators:
        print(f"- {code}: {name}")

if wb_data is not None:
    # Clean data
    wb_data = wb_data.reset_index()
    


    # Prompt for saving the output
    print("Please select where to save the output file...")
    save_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                            filetypes=[("CSV files", "*.csv")])
    
    # Export to CSV
    if save_path:
        wb_data.to_csv(save_path, index=False)
        print(f"Data exported to: {save_path}")
    
    print("\nFirst few rows of downloaded data:")
    print(wb_data.head())
else:
    print("\nNo data could be downloaded. Please check your indicator codes.")

Please select the countries Excel file...


2024-12-07 19:18:46.729 python[31208:2625293] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-07 19:18:47.364 python[31208:2625293] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


Please select the indicators Excel file...

Attempting to download data...
Could not download indicator: AG.AGR.TRAC.NO - Agricultural machinery, tractors
Could not download indicator: AG.LND.TRAC.ZS - Agricultural machinery, tractors per 100 sq. km of arable land
Could not download indicator: EN.ATM.METH.AG.ZS - Agricultural methane emissions (% of total)
Could not download indicator: EN.ATM.METH.AG.KT.CE - Agricultural methane emissions (thousand metric tons of CO2 equivalent)
Could not download indicator: EN.ATM.NOXE.AG.ZS - Agricultural nitrous oxide emissions (% of total)
Could not download indicator: EN.ATM.NOXE.AG.KT.CE - Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent)
Could not download indicator: EN.ATM.CO2E.KD.GD - CO2 emissions (kg per 2015 US$ of GDP)
Could not download indicator: EN.ATM.CO2E.PP.GD.KD - CO2 emissions (kg per 2017 PPP $ of GDP)
Could not download indicator: EN.ATM.CO2E.PP.GD - CO2 emissions (kg per PPP $ of GDP)
Could not downlo

2024-12-07 19:19:52.009 python[31208:2625293] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


Data exported to: /Users//worldbank.csv

First few rows of downloaded data:
               country  date  \
0  Antigua and Barbuda  2023   
1  Antigua and Barbuda  2022   
2  Antigua and Barbuda  2021   
3  Antigua and Barbuda  2020   
4  Antigua and Barbuda  2019   

   Access to clean fuels and technologies for cooking (% of population)  \
0                                                NaN                      
1                                              100.0                      
2                                              100.0                      
3                                              100.0                      
4                                              100.0                      

   Access to clean fuels and technologies for cooking, rural (% of rural population)  \
0                                                NaN                                   
1                                              100.0                                   
2               

: 